In [ ]:
import numpy as np
import os
from os import getcwd as cwd
from os.path import join as pj
import pandas as pd
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.utils.data as data
import visdom

# Logger
from IO.logger import Logger
# Data Sampling
from dataset.classification.sampler import adopt_sampling
# model
from model.optimizer import AdamW

In [ ]:
class args:
    experiment_name = "linearNet_b80_l3_f100_oversample"
    # paths
    bbox_data_path = pj(cwd(), "data/bbox_data", "target_only_20200806.csv")
    figure_root = pj(cwd(), "figure/bbox2size", experiment_name)
    model_root = pj(cwd(), "output_model/bbox2size", experiment_name)
    # model config
    linear_num = 3
    feature_num = 100
    # train config
    bs = 80
    lr = 1e-3
    nepoch = 300
    sampling = "OverSample" # choice [None, "RandomSample", "OverSample"]
    use_WN_L1Loss = False
    # visdom
    visdom = False
    port = 8097

In [ ]:
if os.path.exists(args.figure_root) is False:
    os.makedirs(args.figure_root)

In [ ]:
if torch.cuda.is_available():
    args.cuda = True
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    args.cuda = False
    torch.set_default_tensor_type('torch.FloatTensor')
torch.multiprocessing.set_start_method('spawn')

#### Save args

In [ ]:
args_logger = Logger(args)
args_logger.save()

### visdom

In [ ]:
if args.visdom:
    # Create visdom
    vis = visdom.Visdom(port=args.port)
    
    win_train_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )
    win_test_loss = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='test_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )

In [ ]:
def visualize(vis, phase, visualized_data, window):
    vis.line(
        X=np.array([phase]),
        Y=np.array([visualized_data]),
        update='append',
        win=window
    )

### dataset

In [ ]:
class bbox2size_dataset(data.Dataset):
    
    def __init__(self, bbox_data, size_data=None, mode="test"):
        self.bbox_data = bbox_data
        self.size_data = size_data
        self.mode = mode
        
    def __getitem__(self, index):
        bbox = self.bbox_data[index].astype("float32")
        bbox = torch.from_numpy(bbox)
        
        if self.mode == "train" or self.mode == "eval":
            size = self.size_data[index].astype("float32")
            return bbox, size
        else:
            return bbox
        
    def __len__(self):
        return self.bbox_data.shape[0]

### model

In [ ]:
class LinearNet(nn.Module):
    
    def __init__(self, linear_num=2, feature_num=100):
        super(LinearNet, self).__init__()
        self.linear_num = linear_num
        self.feature_num = feature_num
        
        # define activation function
        self.relu = nn.ReLU(inplace=True)
        
        # create model
        layers = []
        for i in range(linear_num):
            if i == 0:
                linear = nn.Linear(3, feature_num)
                layers += [linear, self.relu]
            elif i == linear_num - 1:
                linear = nn.Linear(feature_num, 1)
                layers += [linear]
            else:
                linear = nn.Linear(feature_num, feature_num)
                layers += [linear, self.relu]
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.model(x)
        return x

### loss function

In [ ]:
class Weight_Normalized_L1loss(nn.L1Loss):
    
    def __init__(self, weight, reduction='mean'):
        super(Weight_Normalized_L1loss, self).__init__(size_average=None, reduce=None, reduction=reduction)
        self.weight = weight
        self.reduction = reduction
        self.l1loss = nn.L1Loss(reduction='none')
        
    def forward(self, output_class, output, target):
        weight = self.weight[output_class]
        if self.reduction == 'mean':
            return torch.mean(weight * self.l1loss(output, target))
        elif self.reduction == 'sum':
            return torch.sum(weight * self.l1loss(output, target))
        else:
            print("error! Weight_Normalized_L1loss.forward")

            
def define_weight(bbox_train):
    labels = bbox_train[:, 2]
    _, count = np.unique(labels, return_counts=True)
    weight = 1 / (count/count.sum())
    return torch.Tensor(weight)

### training

In [ ]:
def train(model, train_dataloader, test_dataloader, lr=1e-4, nepoch=100, visdom=False, use_WN_L1Loss=False):
    # define loss
    if use_WN_L1Loss is True:
        train_l1_loss = Weight_Normalized_L1loss(define_weight(bbox_train))
    else:
        train_l1_loss = nn.L1Loss(reduction='mean')
    test_l1_loss = nn.L1Loss(reduction='mean')
    
    # define optimizer
    opt = AdamW(model.parameters(), lr=lr)
    # set model train mode
    model.train()
    
    # set best loss
    best_total_test_avg_loss = 1e6
    
    for epoch in range(nepoch):
        total_train_loss = 0
        total_test_loss = 0
        # train
        count = 0
        for bbox, size in train_dataloader:
            count += 1
            if args.cuda is True:
                bbox = bbox.cuda()
                size = size.cuda()
            opt.zero_grad()
            out = model(bbox)
            if use_WN_L1Loss is True:
                train_loss = train_l1_loss(bbox[:, 2].long(), out, size[:, None])
            else:
                train_loss = train_l1_loss(out, size[:, None])
            total_train_loss += train_loss.item()
            train_loss.backward()
            opt.step()
        
        total_train_avg_loss = total_train_loss / count
        
        # valid
        model.eval()
        count = 0
        for bbox, size in test_dataloader:
            count += 1
            if args.cuda is True:
                bbox = bbox.cuda()
                size = size.cuda()
            out = model(bbox)
            test_loss = test_l1_loss(out, size[:, None])
            total_test_loss += test_loss.item()
            
        total_test_avg_loss = total_test_loss / count
        model.train()
        
        if total_test_avg_loss < best_total_test_avg_loss:
            best_total_test_avg_loss = total_test_avg_loss
            torch.save(model.state_dict(), pj(args.model_root, "valid_" + str(valid_count) + "_best.pth"))
            with open(pj(args.model_root, "valid_" + str(valid_count) + "_best_loss.txt"), mode="w") as f:
                f.write("epoch = {}, test_loss = {}".format(epoch, total_test_avg_loss))
        
        if visdom:
            visualize(vis, epoch+1, total_train_avg_loss, win_train_loss)
            visualize(vis, epoch+1, total_test_avg_loss, win_test_loss)
        print("epoch=%s: train_loss=%f, test_loss=%f" % (epoch, total_train_avg_loss, total_test_avg_loss))

### estimation

In [ ]:
def estimate_size(model, all_dataloader):
    size_array = []
    
    model.eval()
    for bbox, _ in all_dataloader:
        if args.cuda is True:
            bbox = bbox.cuda()
        out = model(bbox)
        size_array.extend(out[:, 0].cpu().detach().numpy())

    model.train()
    return size_array

In [ ]:
# load data
bbox_df = pd.read_csv(args.bbox_data_path)
bbox_data = np.array(bbox_df.loc[:, ["width", "height", "label"]])
size_data = np.array(bbox_df.loc[:, "size"])
label_data = np.array(bbox_df.loc[:, "label"])
all_dataset = bbox2size_dataset(bbox_data, size_data, mode="eval")
all_dataloader = data.DataLoader(all_dataset, 1, num_workers=0, shuffle=False)

# define kfold
kf = KFold(n_splits=5)
valid_count = 0

# cross validation
total_eval_train_loss = 0
total_eval_test_loss = 0
total_eval_all_loss = 0
for train_index, test_index in kf.split(bbox_data):
    print("")
    valid_count += 1
    print("----- valid {} -----".format(valid_count))
    print("")
    # create validation data
    train_index = adopt_sampling(label_data, train_index, args.sampling)
    bbox_train, bbox_test = bbox_data[train_index], bbox_data[test_index]
    size_train, size_test = size_data[train_index], size_data[test_index]
    # create dataloader
    train_dataset = bbox2size_dataset(bbox_train, size_train, mode="train")
    train_dataloader = data.DataLoader(train_dataset, args.bs, num_workers=0, shuffle=True)
    valid_dataset = bbox2size_dataset(bbox_train, size_train, mode="eval")
    valid_dataloader = data.DataLoader(valid_dataset, 1, num_workers=0, shuffle=False)
    test_dataset = bbox2size_dataset(bbox_test, size_test, mode="eval")
    test_dataloader = data.DataLoader(test_dataset, 1, num_workers=0, shuffle=False)
    
    # create model
    model = LinearNet(linear_num=args.linear_num, feature_num=args.feature_num).cuda()
    
    # training
    train(model, train_dataloader, test_dataloader, lr=args.lr, nepoch=args.nepoch, visdom=args.visdom, use_WN_L1Loss=args.use_WN_L1Loss)
    
    # evaluation
    model.load_state_dict(torch.load(pj(args.model_root, "valid_" + str(valid_count) + "_best.pth")))
    
    estimated_size_array = estimate_size(model, valid_dataloader)
    eval_train_loss = np.sum(np.abs(estimated_size_array - size_train)) / len(size_train)
    total_eval_train_loss += eval_train_loss
    
    estimated_size_array = estimate_size(model, test_dataloader)
    eval_test_loss = np.sum(np.abs(estimated_size_array - size_test)) / len(size_test)
    total_eval_test_loss += eval_test_loss
    
    estimated_size_array = estimate_size(model, all_dataloader)
    eval_all_loss = np.sum(np.abs(estimated_size_array - size_data)) / len(size_data)
    total_eval_all_loss += eval_all_loss
    
    valid_loss = pd.DataFrame({"train": [eval_train_loss], "test": [eval_test_loss], "all": [eval_all_loss]})
    valid_loss.to_csv(pj(args.figure_root, "valid_loss_" + str(valid_count) + ".csv"))
    
    bbox_df_with_estimate_size = pd.DataFrame({"size": size_data})
    bbox_df_with_estimate_size["eval_size"] = estimated_size_array
    bbox_df_with_estimate_size.to_csv(pj(args.figure_root, "output_size_" + str(valid_count) + ".csv"))

In [ ]:
valid_loss = pd.DataFrame({"train": [total_eval_train_loss / 5], "test": [total_eval_test_loss / 5], "all": [total_eval_all_loss / 5]})
valid_loss.to_csv(pj(args.figure_root, "final_loss_" + str(valid_count) + ".csv"))